In [ ]:
# !pip -q uninstall -y evidently
!pip -q install --no-cache-dir "evidently==0.6.7"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.7/565.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 144.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 159.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.1 MB/s eta 0:00:00


In [ ]:
!pip install h2o -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 MB 1.0 MB/s eta 0:00:00


In [ ]:
# Imports

import pandas as pd
import numpy as np
import h2o

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.ui.workspace.cloud import CloudWorkspace

In [ ]:
# Initalize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.17" 2025-10-21; OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04); OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.12/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7713lfyl
  JVM stdout: /tmp/tmp7713lfyl/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp7713lfyl/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_unknownUser_ogu3ru
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.147 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# Evidently Cloud Workspace

from evidently.ui.workspace import CloudWorkspace

ws = CloudWorkspace(token="TOKEN", url="https://app.evidently.cloud")

In [ ]:
# Evidently Project Workspace
project = ws.get_project("PROJECT_ID")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
## Load datasets and model
import pandas as pd
reference_data = pd.read_csv('/content/drive/MyDrive/df_test.csv')
changed_data = pd.read_csv('/content/drive/MyDrive/changed_df_test.csv')

loaded_model = h2o.load_model('/content/drive/MyDrive/BTC_Forecasting_Models/DeepLearning_grid_1_AutoML_11_20251211_223625_model_3')

In [36]:
reference_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   close              573 non-null    float64
 1   volume             573 non-null    int64  
 2   day_of_week        573 non-null    int64  
 3   month              573 non-null    int64  
 4   is_weekend         573 non-null    int64  
 5   sin_dow            573 non-null    float64
 6   cos_dow            573 non-null    float64
 7   sin_month          573 non-null    float64
 8   cos_month          573 non-null    float64
 9   return_log_lag_1   573 non-null    float64
 10  return_log_lag_2   573 non-null    float64
 11  return_log_lag_3   573 non-null    float64
 12  range_hl           573 non-null    float64
 13  close_lag_1        573 non-null    float64
 14  close_lag_2        573 non-null    float64
 15  close_lag_3        573 non-null    float64
 16  close_lag_4        573 non

In [37]:
reference_data.columns

Index(['close', 'volume', 'day_of_week', 'month', 'is_weekend', 'sin_dow',
       'cos_dow', 'sin_month', 'cos_month', 'return_log_lag_1',
       'return_log_lag_2', 'return_log_lag_3', 'range_hl', 'close_lag_1',
       'close_lag_2', 'close_lag_3', 'close_lag_4', 'close_lag_5',
       'close_lag_6', 'close_lag_7', 'target_next_close', 'volatility_7d',
       'momentum_7d', 'rolling_max_7d', 'rolling_min_7d', 'volatility_14d',
       'momentum_14d', 'rolling_max_14d', 'rolling_min_14d', 'volatility_30d',
       'momentum_30d', 'rolling_max_30d', 'rolling_min_30d'],
      dtype='object')

In [38]:
reference_data = reference_data.rename(columns={"close": "Close"})
changed_data   = changed_data.rename(columns={"close": "Close"})
reference_data = reference_data.rename(columns={"volume": "Volume"})
changed_data   = changed_data.rename(columns={"volume": "Volume"})

In [39]:
cols = [
    'Close', 'Volume', 'day_of_week', 'month', 'is_weekend', 'sin_dow',
    'cos_dow', 'sin_month', 'cos_month', 'return_log_lag_1',
    'return_log_lag_2', 'return_log_lag_3', 'range_hl', 'close_lag_1',
    'close_lag_2', 'close_lag_3', 'close_lag_4', 'close_lag_5',
    'close_lag_6', 'close_lag_7', 'volatility_7d', 'momentum_7d',
    'rolling_max_7d', 'rolling_min_7d', 'volatility_14d', 'momentum_14d',
    'rolling_max_14d', 'rolling_min_14d', 'volatility_30d', 'momentum_30d',
    'rolling_max_30d', 'rolling_min_30d'
]

# target column
target_col = "target_next_close"

In [40]:
# Build X/y and predict (force H2O numeric)
from sklearn.metrics import r2_score, mean_squared_error

def metrics(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    print(f"{name}: R2={r2:.4f}, RMSE={rmse:.4f}")

X_ref = reference_data[cols].copy()
y_ref = reference_data[target_col].copy()

X_test = changed_data[cols].copy()
y_test = changed_data[target_col].copy()

# H2OFrames
h2o_ref  = h2o.H2OFrame(X_ref)
h2o_test = h2o.H2OFrame(X_test)

# Force numeric (prevents enum/categorical inference)
for c in cols:
    h2o_ref[c]  = h2o_ref[c].asnumeric()
    h2o_test[c] = h2o_test[c].asnumeric()

y_pred_ref  = loaded_model.predict(h2o_ref).as_data_frame().iloc[:, 0].values
y_pred_test = loaded_model.predict(h2o_test).as_data_frame().iloc[:, 0].values

metrics("BASELINE (changed_data)", y_test.values, y_pred_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.12/dist-packages/h2o/frame.py:1983: H2ODependencyWarning:

Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)




deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
BASELINE (changed_data): R2=0.9781, RMSE=4780.4816


/usr/local/lib/python3.12/dist-packages/h2o/frame.py:1983: H2ODependencyWarning:

Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)




In [ ]:
# Build and send the Evidently snapshot
def send_snapshot(name, X_ref, y_ref, y_pred_ref, X_cur, y_cur, y_pred_cur, ws, project_id):
    ref_df = X_ref.copy()
    cur_df = X_cur.copy()

    ref_df["target"] = np.asarray(y_ref)
    ref_df["prediction"] = np.asarray(y_pred_ref)

    cur_df["target"] = np.asarray(y_cur)
    cur_df["prediction"] = np.asarray(y_pred_cur)

    rpt = Report(metrics=[DataDriftPreset(), RegressionPreset()])
    rpt.run(reference_data=ref_df, current_data=cur_df)

    ws.add_report(project_id, rpt)
    print(f"Sent monitoring snapshot: {name}")

# Send baseline snapshot: reference vs changed
send_snapshot(
    name="baseline_ref_vs_changed",
    X_ref=X_ref, y_ref=y_ref, y_pred_ref=y_pred_ref,
    X_cur=X_test, y_cur=y_test, y_pred_cur=y_pred_test,
    ws=ws, project_id=project.id
)

***Local Evidently Dashboards***

In [41]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from IPython.display import IFrame

def build_local_report(name, X_ref, y_ref, y_pred_ref, X_cur, y_cur, y_pred_cur):
    ref_df = X_ref.copy()
    cur_df = X_cur.copy()

    ref_df["target"] = np.asarray(y_ref)
    ref_df["prediction"] = np.asarray(y_pred_ref)

    cur_df["target"] = np.asarray(y_cur)
    cur_df["prediction"] = np.asarray(y_pred_cur)

    rpt = Report(metrics=[DataDriftPreset(), RegressionPreset()])
    rpt.run(reference_data=ref_df, current_data=cur_df)

    html_name = f"evidently_{name}.html"
    rpt.save_html(html_name)
    print(f"Saved Evidently report: {html_name}")
    return html_name

In [43]:
baseline_html = build_local_report(
    "baseline_ref",
    X_ref,  y_ref,  y_pred_ref,
    X_ref,  y_ref,  y_pred_ref
)

IFrame(baseline_html, width=1200, height=800)

Saved Evidently report: evidently_baseline_ref.html


In [44]:
from google.colab import files
files.download("evidently_baseline_ref.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
baseline_to_changed_html = build_local_report(
    "baseline_ref_vs_changed",
    X_ref,  y_ref,  y_pred_ref,
    X_test, y_test, y_pred_test
)

IFrame(baseline_html, width=1200, height=800)

Saved Evidently report: evidently_baseline_ref_vs_changed.html


In [48]:
from google.colab import files
files.download("evidently_baseline_ref_vs_changed.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>